In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [ ]:
def func(x, a, b, c, d):
    res=np.array([])
    for x_single in x:
        if x_single <= d:
            res= np.append(res, [c])
        else:
            res = np.append(res, [a *(1 - np.exp(-b * (x_single-d))) + c])
    return res

In [ ]:
def tn_pt2(xdata, td, k, w0, d):
    def _a1_2(w0, d):
        return (-d * w0 + w0 * np.sqrt(d**2 - 1), 
                -d * w0 - w0 * np.sqrt(d**2 - 1))
    def _T1_2(a1_2):
        #print a1_2
        a1, a2 = a1_2
        return (-1.0/a1, -1.0/a2)
    def T1_2(w0, d):
        return _T1_2(_a1_2(w0, d))

    res = np.array([])
    for x in xdata:
        if x < td:
            res = np.append(res, [0.0])  
        else:
            t = x - td
            T1, T2 = T1_2(w0, d)
            if d > 1: # Kriechfall
                val = k * (1 - (T1 / (T1 - T2)) * np.exp(-t / T1) + (T2 / (T1 - T2)) * np.exp(-t / T2))
                res = np.append(res, [val])  
            elif d == 1: #aperiodischer Grenzfall
                val = k * (1 - np.exp(-t/T1) - 1.0/T1 * np.exp(-t/T2))
                res = np.append(res, [val])
            elif d <= -1: # instabiler Kriechfall
                val = k * (1 - (T1 / (T1 - T2)) * np.exp(t / T1) + (T2 / (T1 - T2)) * np.exp(t / T2))
                res = np.append(res, [val])
            else: #Schwingfall
                #we = wo * np.sqrt(1-d**2)
                val = k * (1 - 1/(np.sqrt(1-d**2)) * np.exp(-d*w0*t) * np.sin(w0*np.sqrt(1-d**2) * t + np.arccos(d)))
                res = np.append(res, [val])
    return res

In [ ]:
xdata = np.linspace(0, 70, 1000)
y = tn_pt2(xdata, td=0, k=2, w0=1, d=1)
#y = func(xdata, 10, 5, 0, 1)
#print len(y), len(xdata)
y_noise = 0.1 * np.random.normal(size=xdata.size)
#ydata = y + y_noise
#ydata = tn_pt2(xdata, 0, 2, 1, 5)
plt.plot(xdata, y, 'g-', label='data')
#plt.plot(xdata, ydata, 'b-', label='data_noise')
tn_pt2([0], td=0, k=2, w0=1, d=1) ###?????? TODO Correct

In [ ]:
popt, pcov = curve_fit(tn_pt2, xdata, ydata, method='dogbox')
plt.plot(xdata, tn_pt2(xdata, *popt), 'r-', label='fit')
popt

In [ ]:
popt, pcov = curve_fit(tn_pt2, xdata, ydata, method='dogbox', bounds=(0, [15., 10., 1., 0.2]))
plt.plot(xdata, tn_pt2(xdata, *popt), 'b--', label='fit-with-bounds')
popt

In [ ]:
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()
